<a href="https://colab.research.google.com/github/RithikaReddy27/Competitor-Strategy-Tracker-ECommerce/blob/main/APP_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 437.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 242.7 kB/s eta 0:00:00


In [4]:
import pandas as pd
import plotly.express as px
import requests
import streamlit as st
from openai import AzureOpenAI
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from transformers import pipeline
from datetime import datetime
import json

In [5]:
# API_KEY.py

GROQ_API_KEY = "your_groq_api_key_here"
SLACK_WEBHOOK_API_KEY = "your_slack_webhook_api_key_here"


In [6]:
%%writefile API_KEY.py
GROQ_API_KEY = "your_groq_api_key_here"
SLACK_WEBHOOK_API_KEY = "your_slack_webhook_api_key_here"


Writing API_KEY.py


In [7]:

from API_KEY import GROQ_API_KEY
from API_KEY import SLACK_WEBHOOK_API_KEY


In [8]:
def truncate_text(text, max_length=512):
    return text[:max_length]

In [9]:
def load_competitor_data():
    # load competitor data from competitor_data.csv
    data = pd.read_csv("competitor_data.csv")
    print("competitor data loaded")
    print(data.head())
    return data

In [10]:
def load_reviews_data():
    # load reviews data from reviews_data.csv
    reviews = pd.read_csv("reviews.csv")
    print("reviews data loaded")
    print(reviews.head())
    return reviews

In [11]:
def analyze_sentiment(reviews):
    # analyze sentiment of text
    sentiment_pipeline = pipeline("sentiment-analysis")
    return sentiment_pipeline(reviews)

In [12]:
def train_predictive_model(data):
    data['discount'] = data['discount'].str.replace('%', '').astype(float)
    data['price'] = data['price'].str.replace('$', '').astype(float)
    data['predicted_discount'] = data['discount'] + (data['price'] * 0.05).round(2)

    X = data[['price', 'discount']]
    y = data['predicted_discount']
    print(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
        )

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    return model

In [13]:
import numpy as np
import pandas as pd

def forecast_discounts_arima(data,future_days = 5):
    """
    Forcast future discounts using ARIMA model
    :param data: pandas DataFrame historical discount data with datetime index
    :param future_days: int number of days to forecast
    :return: pandas DataFrame with historical and forecasted discount data
    """

    data = data.sort_index()
    print(product_data.index)

    data['discount'] = pd.to_numeric(data['discount'], errors='coerce')
    data = data.dropna(subset=['discount'])

    discount_series = data['discount']
    if not isinstance(data.index, pd.DatetimeIndex):
        try:
            data.index = pd.to_datetime(data.index)
        except Exception as e:
            raise ValueError(
                "Index could not be converted to datetime"
                ) from e

    model = ARIMA(discount_series, order=(5,1,0))
    model_fit = model.fit()

    forecast = model_fit.forecast(steps=future_days)
    future_dates = pd.date_range(
        start=discount_series.index[-1] + pd.Timedelta(days=1), periods=future_days
    )

    forecast_df = pd.DataFrame({'date': future_dates, 'predicted_discount': forecast})
    forecast_df.set_index('date', inplace=True)

    return forecast_df


In [14]:
def send_to_slack(data):
    # Send message to slack

    payload = {"text": data}
    response = requests.post(
        SLACK_WEBHOOK,
        data=json.dumps(payload),
        headers={"Content-Type": "application/json"},
    )

In [15]:
def generate_strategy_recommendation(product_name, competitor_data, sentiment):
    # generate strategy recommendation using an LLM model
    date = datetime.now()
    prompt = f"""
    You are a highly skilled business strategist specilizing in e-commerce.
    Based on the following details,
    suggest actionable strategies to optimize pricing,
    promotions and customer satisfaction
    for the selected product

    1. Product Name: {product_name}

    2. Competitor Data (inclusing current prices, discounts and predicted discounts):
    {competitor_data}

    3. Sentiment Analysis: {sentiment}

    4. Today's Date: {str(date)}


    ### Task:
    - Analyze the competitor data and identify key pricing trends.
    - Leverage the sentiment analysis to higlight areas where customer satisfaction can be improved.
    - Use the discount predictions to suggest how pricing strategies can be optimized over next 5 days.
    - Recommend promotional campaigns or marketing strategies that align with customer sentiment and competitive trends.
    - Ensure the strategies are actionable, realistic, and geared towards increasing customer satisfaction, driving sales and maximizing profits.

    Provide your recommendations in a structured format:
    1. Pricing Strategy
    2. Promotional Campaign Ideas
    3. Customer Satisfaction Recommendations

    """

    message = [{"role":"user","content":prompt}]

    data = {
        "messages": message,
        "model": "llama3-8b-8192",
        "temperature" : 0
    }

    headers = {"Context-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}

    res = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        data = json.dumps(data),
        headers=headers,
    )

    print("API Response:", res)

    res = res.json()


    response = res['choices'][0]['message']['content']
    return response



In [16]:
import pandas as pd
import streamlit as st
import plotly.express as px
from textblob import TextBlob

# Helper functions
def load_competitor_data():
    # Load competitor data from a file
    return pd.DataFrame({
        "product_name": ["boAt Rockerz 255", "Oneplus Bullets Z2", "Realme Buds Wireless 3 Neo", "JBL Tune 215BT"],
        "price": [1500, 1800, 1600, 2000],
        "rating": [4.2, 4.5, 4.0, 4.3],
    })

def load_reviews():
    # Load reviews data from a file
    return pd.DataFrame({
        "product_name": ["boAt Rockerz 255", "Oneplus Bullets Z2", "boAt Rockerz 255"],
        "reviews": ["Great sound quality!", "Battery life could be better.", "Excellent value for money!"]
    })

def truncate_text(text, max_length):
    return text[:max_length] + "..." if len(text) > max_length else text

def analyze_sentiment(reviews):
    sentiments = [{"text": review, "label": "Positive" if TextBlob(review).sentiment.polarity > 0 else "Negative"} for review in reviews]
    return sentiments

# Streamlit app
st.set_page_config(page_title="E-commerce Competitor Strategy Dashboard", layout="wide")
st.title("E-commerce Competitor Strategy Dashboard")
st.sidebar.header("Select a Product")

products = [
    "boAt Rockerz 255",
    "Oneplus Bullets Z2",
    "Realme Buds Wireless 3 Neo",
    "JBL Tune 215BT"
]

selected_product = st.sidebar.selectbox("Select a Product to analyze", products)

competitor_data = load_competitor_data()
reviews_data = load_reviews()

product_data = competitor_data[competitor_data['product_name'] == selected_product]
product_reviews = reviews_data[reviews_data['product_name'] == selected_product]

st.header(f"Competitor Analysis for {selected_product}")
st.subheader("Competitor Data")
st.table(product_data.tail(5))

if not product_reviews.empty:
    product_reviews["reviews"] = product_reviews["reviews"].apply(lambda x: truncate_text(x, 512))
    reviews = product_reviews["reviews"].tolist()
    sentiments = analyze_sentiment(reviews)

    st.subheader("Customer Sentiment Analysis")
    sentiment_df = pd.DataFrame(sentiments)

    fig = px.bar(sentiment_df, x='label', title='Sentiment Analysis Results')
    st.plotly_chart(fig)

else:
    st.write("No reviews data available for this product.")


2025-02-19 11:27:45.590 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:27:45.591 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:27:45.870 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-19 11:27:45.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:27:45.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:27:45.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:27:45.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [17]:
import pandas as pd

# Generate a sequential date column (example: daily dates starting from today)
product_data['date'] = pd.date_range(start="2025-01-01", periods=len(product_data), freq="D")

product_data.set_index("date", inplace=True)
product_data = product_data.sort_index()


<ipython-input-17-9e20dcb8b045>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
# Check for the 'date' column
if 'date' in product_data.columns:
    product_data['date'] = pd.to_datetime(product_data['date'], errors='coerce')
    product_data = product_data.dropna(subset=['date'])
    product_data.set_index("date", inplace=True)
    product_data = product_data.sort_index()
else:
    print("'date' column is missing. Skipping date-related processing.")

# Process 'discount' column
if 'discount' in product_data.columns:
    product_data['discount'] = pd.to_numeric(product_data['discount'], errors='coerce')
    product_data = product_data.dropna(subset=['discount'])
else:
    print("Column 'discount' does not exist in product_data.")


'date' column is missing. Skipping date-related processing.
Column 'discount' does not exist in product_data.


In [19]:
!wget -q -O - ipv4.icanhazip.com

35.188.252.251


In [20]:
import pandas as pd

# Generate a sequential date column starting from a fixed date
start_date = "2025-01-01"  # Replace with your desired start date
product_data['date'] = pd.date_range(start=start_date, periods=len(product_data), freq="D")

product_data.set_index("date", inplace=True)
product_data = product_data.sort_index()

print(product_data.head())


                product_name  price  rating
date                                       
2025-01-01  boAt Rockerz 255   1500     4.2


In [21]:
%%writefile app.py
import streamlit as st

st.title("My Streamlit App")
st.write("Hello, welcome to my app!")


Writing app.py


In [23]:
%%writefile app.py
import streamlit as st

st.title("My Streamlit App")
st.write("Hello, welcome to my app!")


Overwriting app.py


In [24]:
! pip install streamlit -q

In [25]:
!wget -q -O - ipv4.icanhazip.com

35.188.252.251


In [26]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Title of the dashboard
st.title("E-commerce Competitor Strategy Dashboard")

# File upload section
uploaded_file = st.sidebar.file_uploader("Upload a CSV file", type=["csv"])

if uploaded_file:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(uploaded_file)

    # Ensure necessary columns exist in the file
    required_columns = {"product_name", "price", "discount", "date"}
    if not required_columns.issubset(df.columns):
        st.error(f"CSV file must contain the following columns: {', '.join(required_columns)}")
    else:
        # Sidebar for product selection
        st.sidebar.header("Select a Product")
        selected_product = st.sidebar.selectbox(
            "Select a Product to analyze", df["product_name"].unique()
        )

        # Competitor Analysis
        st.subheader(f"Competitor Analysis for {selected_product}")
        product_data = df[df["product_name"] == selected_product]
        st.table(product_data)

        # Customer Sentiment Analysis (Simulated data)
        st.subheader("Customer Sentiment Analysis")
        # Simulate sentiment data dynamically (replace with real analysis if available)
        sentiment_data = {"NEGATIVE": 25, "POSITIVE": 25}

        # Plotting Sentiment Analysis Results
        fig, ax = plt.subplots()
        ax.bar(sentiment_data.keys(), sentiment_data.values(), color="skyblue")
        ax.set_title("Sentiment Analysis Results")
        ax.set_ylabel("Count")
        ax.set_xlabel("Sentiment")

        # Display Sentiment Analysis Plot
        st.pyplot(fig)

        # Product Recommendations
        st.subheader("Product Recommendations")

        # Generate Recommendations (based on price and discount)
        recommendation_data = df[df["product_name"] != selected_product]
        recommendations = recommendation_data.sort_values(
            by=["discount", "price"], ascending=[False, True]
        ).head(3)

        if not recommendations.empty:
            recommendations = recommendations[["product_name", "price", "discount"]]
            recommendations.rename(
                columns={
                    "product_name": "Recommended Product",
                    "price": "Price",
                    "discount": "Discount (%)",
                },
                inplace=True,
            )
            st.write("Based on your selected product, here are some recommendations:")
            st.table(recommendations)
        else:
            st.write("No recommendations available.")
else:
    st.write("Please upload competitor_data.csv to begin analysis.")

2025-02-19 11:29:22.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.264 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:29:22.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.188.252.251:8501

⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://major-years-cover.loca.lt
